# Gallery Example: Hyper/Erl/1 Feedback Queue

This example demonstrates a HyperExp/Erlang/1 queueing system with feedback routing:
- **Arrivals**: Hyperexponential distribution (high variability)
- **Service**: Erlang distribution (low variability)
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Feedback**: Jobs may return to the queue with some probability

In feedback queues, jobs may loop back and require additional service.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_hyperl1_feedback(): Network {
    """Create Hyper/Erl/1-Feedback queueing model"""
    val model = Network("Hyper/Erl/1-Feedback")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
// HyperExp arrival (SCV=64)
    source.setArrival(oclass1, HyperExp.fit_mean_and_scv(1, 64))
// Erlang service
    queue.setService(oclass1, Erlang.fit_mean_and_order(0.5, 5))
// Block 3: topology with feedback (90% feedback probability)
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addRoute(oclass1, queue, queue, 0.9)  # 90% loop back
    P.addRoute(oclass1, queue, sink, 0.1)   # 10% exit
    model.link(P)
    
    return model
// Create the model
val model = gallery_hyperl1_feedback()

## About Feedback Queues

In feedback queues:
- Jobs may require multiple passes through the server
- After each service, there's a probability of returning
- Mean number of visits = 1 / (1 - feedback_probability)

With 90% feedback probability:
- Mean visits per job = 1 / 0.1 = 10
- Effective arrival rate at queue = external_rate × mean_visits
- Must ensure stability: effective_rate < service_rate

This creates high utilization and potential congestion.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)